### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

In [2]:
# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [3]:
# number of times a metabolite apperas on pairs dataset
graph.get_number_of_occurences(pairs)
graph.create_graph(data=data, pairs=pairs)

# nodes: 8591 
# edges: 30026


100%|██████████| 8591/8591 [00:00<00:00, 443388.12it/s]


In [4]:
graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

  0%|          | 0/30026 [00:00<?, ?it/s]

  1%|▏         | 378/30026 [00:00<00:20, 1478.86it/s][15:31:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:31:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:31:49] WARNING: not removing hydrogen atom without neighbors
  3%|▎         | 768/30026 [00:00<00:18, 1590.95it/s][15:31:49] WARNING: not removing hydrogen atom without neighbors
[15:31:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:31:49] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:31:49] WARNING: not removing hydrogen atom with dummy atom neighbors
 11%|█▏        | 3378/30026 [00:02<00:23, 1139.88it/s][15:31:51] WARNING: not removing hydrogen atom without neighbors
[15:31:51] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:31:51] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:31:51] WARNING: not removing hydrogen atom without neighbors
 13%|█▎        | 3885/30026 [00:02<00:18, 1431.69it/s][15:31:51] WARN

In [5]:
p, idx = graph.constrained_shortest_path('C00082', 'C01533', weight='mol_weight')

In [6]:
p[idx]

['C00082',
 'C00811',
 'C01197',
 'C01494',
 'C05619',
 'C00482',
 'C05610',
 'C02325',
 'C01533']

In [11]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))
paths = graph.validate(test_cases, 'mol_weight')

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [01:18<00:00,  1.56s/it]

Correct pathway predictions: 19
Correct pathway predictions (%): 38.0


In [12]:
paths

,Pathway,Correct
0,"[C00082, C00811, C01197, C01494, C05619, C0048...",True
1,"[C00223, C12096, C00029, C00761]",False
2,"[C00811, C01197, C01494, C05619, C00482, C0117...",True
3,"[C00079, C00423, C00540, C00903]",True
4,"[C00223, C12096, C00029, C17750]",False
5,"[C06561, C00509, C16492, C01460]",True
6,"[C05903, C12249, C12634, C12635, C12636]",True
7,"[C01477, C01514, C03515, C12632, C04900]",True
8,"[C05903, C00389, C10107, C12633, C11620]",True
9,"[C05905, C08604, C12095, C12096, C16299]",False


In [14]:
test_cases.tail()

,Pathway,source,target,paths_list
45,"C16391,C16392,C16394,C16396,C16400",C16391,C16400,"[C16391, C16392, C16394, C16396, C16400]"
46,"C07083,C07084,C07085,C07087,C00511,C00894,C008...",C07083,C00186,"[C07083, C07084, C07085, C07087, C00511, C0089..."
47,"C11249,C00854,C00414,C01880,C06103,C06102,C061...",C11249,C02232,"[C11249, C00854, C00414, C01880, C06103, C0610..."
48,"C00829,C04314,C03012,C06204,C06203,C06202,C00805",C00829,C00805,"[C00829, C04314, C03012, C06204, C06203, C0620..."
49,"C07535,C18270,C18279,C18277,C18283",C07535,C18283,"[C07535, C18270, C18279, C18277, C18283]"


In [8]:
graph.G.edges(data=True)

EdgeDataView([('C00013', 'C00009', {'mol_weight': inf, 'smiles_similarity': 0.74}), ('C00013', 'C00044', {'mol_weight': inf, 'smiles_similarity': 0.9610894941634242}), ('C00013', 'C00002', {'mol_weight': inf, 'smiles_similarity': 0.9590834697217676}), ('C00013', 'C00001', {'mol_weight': inf, 'smiles_similarity': 1.0}), ('C00013', 'C00455', {'mol_weight': inf, 'smiles_similarity': 0.9667036625971143}), ('C00013', 'C00080', {'mol_weight': inf, 'smiles_similarity': 1.0}), ('C00013', 'C00536', {'mol_weight': inf, 'smiles_similarity': 0.5454545454545454}), ('C00013', 'C00061', {'mol_weight': inf, 'smiles_similarity': 0.9721804511278196}), ('C00013', 'C00073', {'mol_weight': inf, 'smiles_similarity': 0.9798657718120806}), ('C00013', 'C00857', {'mol_weight': inf, 'smiles_similarity': 0.9627699180938198}), ('C00013', 'C00014', {'mol_weight': inf, 'smiles_similarity': 1.0}), ('C00013', 'C00147', {'mol_weight': inf, 'smiles_similarity': 0.9737609329446064}), ('C00013', 'C00119', {'mol_weight': i

In [9]:
1/0

ZeroDivisionError: division by zero

In [ ]:
graph.calculate_smiles_similarity(data)

 38%|███▊      | 11288/30026 [00:01<00:02, 7489.04it/s][14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
[14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
[14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
[14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
[14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
[14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
[14:55:09] WARNING: not removing hydrogen atom with dummy atom neighbors
100%|██████████| 30026/30026 [00:49<00:00, 602.33it/s] 


In [ ]:
p = graph.constrained_shortest_path('C00082', 'C01533', weight='mol_weigh')
p

KeyError: 'smiles_similarity'

In [ ]:
from itertools import chain

def has_duplicates(lst):
    return len(lst) != len(set(lst))

p = graph.shortest_simple_paths('C00082', 'C01533', weight='mol_weight')

smiles_sim = []

for j in p:
    rxns_list = []
    sum = 0
    compound_list = j
    for i in range(len(compound_list) - 1):
        current_compound = compound_list[i]
        next_compound = compound_list[i + 1]
        sum = sum + graph.G.edges()[current_compound, next_compound]['smiles_similarity']
        if graph.G.edges()[current_compound, next_compound]['smiles_similarity'] == 0:
            sum = sum + graph.G.edges()[next_compound, current_compound]['smiles_similarity']
        rxns_list.append(pairs[(pairs['source']==current_compound) & (pairs['target']==next_compound)]['Reaction'].values)
        rxns_list.append(pairs[(pairs['source']==next_compound) & (pairs['target']==current_compound)]['Reaction'].values)

    smiles_sim.append(sum)
    rxns_list = [arr for arr in rxns_list if arr.any()]
    rxns_list = list(chain.from_iterable(rxns_list))
    has_duplicate = has_duplicates(rxns_list)
    print(has_duplicate)

False
True
False
False
True
False
True
False
True
False


In [ ]:
smiles_sim.index(min(smiles_sim))

3

In [ ]:
idx = smiles_sim.index(min(smiles_sim))

print(idx)

In [ ]:
from itertools import islice

next(islice(p, idx, None))

In [ ]:
test_cases['Pathway '].iloc[0]

In [ ]:
paths.iloc[0]['Pathway']

In [ ]:
1/0

In [ ]:
for i in graph.shortest_simple_paths('C00082', 'C01533', length=10, weight='mol_weight'):
    print(i)

In [ ]:
test_cases

### Try cluster graph nodes

### Try similarity based on SMILES

In [ ]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))